In [7]:
import sqlite3
import pandas as pd

# File path for the CSV
csv_file_path = 'data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'


sqlite_db_path = "police_reports.db"


table_name = "police_incident_reports"

try:
    df = pd.read_csv(csv_file_path)
    print("CSV file successfully loaded into a DataFrame.")
except Exception as e:
    print(f"Error loading CSV file: {e}")


try:
    conn = sqlite3.connect(sqlite_db_path)
    print(f"SQLite database connected. Database path: {sqlite_db_path}")

    
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Data successfully written to table '{table_name}' in the SQLite database.")

    
    query = f"""SELECT *,
    CASE 
        WHEN Category IN ('KIDNAPPING', 'SEX OFFENSES, FORCIBLE', 'WEAPON LAWS', 
                          'ARSON', 'ROBBERY', 'ASSAULT', 'DRUG/NARCOTIC', 
                          'PROSTITUTION', 'EXTORTION', 'BURGLARY', 'VEHICLE THEFT') THEN 'High'
        WHEN Category IN ('SUSPICIOUS OCC', 'FORGERY/COUNTERFEITING', 'OTHER OFFENSES', 
                          'STOLEN PROPERTY', 'TRESPASS', 'MISSING PERSON', 'WARRANTS', 
                          'DRIVING UNDER THE INFLUENCE', 'TREA', 'EMBEZZLEMENT', 
                          'DRUNKENNESS', 'SECONDARY CODES', 'RECOVERED VEHICLE', 
                          'LARCENY/THEFT', 'SUICIDE', 'FRAUD', 'PORNOGRAPHY/OBSCENE MAT', 
                          'LOITERING', 'BAD CHECKS', 'LIQUOR LAWS', 'GAMBLING', 
                          'NON-CRIMINAL', 'DISORDERLY CONDUCT', 'BRIBERY', 'VANDALISM',
                          'SEX OFFENSES, NON FORCIBLE') THEN 'Low'
        ELSE 'Low'
    END AS Crime_Level
    FROM {table_name};"""
    print(f"\\nExecuting query:\\n{query}")

    
    result_df = pd.read_sql_query(query, conn)

    
    print("\\nQuery Results:")
    print(result_df)

except Exception as e:
    print(f"Error setting up SQLite connection, writing data, or querying data: {e}")
finally:
    if 'conn' in locals() or 'conn' in globals():
        conn.close()
        print("SQLite connection closed.")

Error loading CSV file: [Errno 2] No such file or directory: 'data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'
SQLite database connected. Database path: police_reports.db
Data successfully written to table 'police_incident_reports' in the SQLite database.
\nExecuting query:\nSELECT *,
    CASE 
        WHEN Category IN ('KIDNAPPING', 'SEX OFFENSES, FORCIBLE', 'WEAPON LAWS', 
                          'ARSON', 'ROBBERY', 'ASSAULT', 'DRUG/NARCOTIC', 
                          'PROSTITUTION', 'EXTORTION', 'BURGLARY', 'VEHICLE THEFT') THEN 'High'
        WHEN Category IN ('SUSPICIOUS OCC', 'FORGERY/COUNTERFEITING', 'OTHER OFFENSES', 
                          'STOLEN PROPERTY', 'TRESPASS', 'MISSING PERSON', 'WARRANTS', 
                          'DRIVING UNDER THE INFLUENCE', 'TREA', 'EMBEZZLEMENT', 
                          'DRUNKENNESS', 'SECONDARY CODES', 'RECOVERED VEHICLE', 
                          'LARCENY/THEFT', 'SUICIDE', 'FRAUD', 'PORNOGRAPHY/OBSCENE M

In [56]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler

In [12]:
os.getcwd()

'/Users/kandicetran/Downloads'

In [70]:
df = pd.read_csv("/Users/kandicetran/Downloads/Police_Department_Incident_Reports__Historical_2003_to_May_2018_20250930.csv")

In [71]:
df = df.dropna() # Drop row with null values
df.isnull().sum() # Double check null values

PdId              0
IncidntNum        0
Incident Code     0
Category          0
Descript          0
DayOfWeek         0
Date              0
Time              0
PdDistrict        0
Resolution        0
Address           0
X                 0
Y                 0
location          0
data_loaded_at    0
dtype: int64

In [72]:
# Drop last four columns: X, Y, location, data_loaded_at (not needed for analysis)
df = df.iloc[:, :-2]

Add Additional Part of Analysis below here

In [73]:
# Transform date and timestamp columns to month, year, and hour of day
df['Date'] = pd.to_datetime(df['Date'])  # Double check Date is in timeframe format
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

df['Time'] = pd.to_datetime(df['Time']) # Double check Time is in timestamp format
df['Hour'] = df['Time'].dt.hour 
df['Season'] = df['Month'].map({12:'Winter',1:'Winter',2:'Winter',
                                3:'Spring',4:'Spring',5:'Spring',
                                6:'Summer',7:'Summer',8:'Summer',
                                9:'Fall',10:'Fall',11:'Fall'})

df = df.drop(['Date', 'Time','IncidntNum','Incident Code','Address'], axis=1) # Drop redundant columns
df.head()

,PdId,Category,Descript,DayOfWeek,PdDistrict,Resolution,X,Y,Year,Month,Hour,Season
0,11049313327195,TRESPASS,TRESPASSING,Sunday,TARAVAL,NONE,-122.466758,37.729185,2011,6,13,Summer
1,6102672004134,ASSAULT,BATTERY,Monday,NORTHERN,NONE,-122.425839,37.778486,2006,9,22,Fall
2,16020415607021,VEHICLE THEFT,STOLEN AUTOMOBILE,Thursday,TARAVAL,NONE,-122.463545,37.707968,2016,3,19,Spring
3,13011910904134,ASSAULT,BATTERY,Sunday,MISSION,NONE,-122.425237,37.754222,2013,2,18,Winter
4,15021632104134,ASSAULT,BATTERY,Tuesday,SOUTHERN,NONE,-122.403405,37.775421,2015,3,0,Spring


In [74]:
# Data formatting
df['Category'] = df['Category'].str.title().str.strip()
df['Descript'] = df['Descript'].str.title().str.strip()
df['PdDistrict'] = df['PdDistrict'].str.title().str.strip()
df['Resolution'] = df['Resolution'].str.title().str.strip()
df['Month'] = df['Month'].astype(int)
df['Year'] = df['Year'].astype(int)
df['Hour'] = df['Hour'].astype(int)

#Extract coordinates for spatial clustering
coords = df[['X', 'Y']]
scaled_coords = StandardScaler().fit_transform(coords)
scaled_coords

array([[-1.47509049, -0.09876849],
       [-0.10317343,  0.01670602],
       [-1.36736536, -0.14846105],
       ...,
       [ 0.28474971,  0.03010745],
       [-0.39568048,  0.02997249],
       [-0.14026403,  0.01906136]])

In [78]:
# Define the output CSV file name
output_file = 'SF_Police_Reports_2003_to_2018_cleaned.csv'
df.to_csv(output_file, index=False) # index=False prevents writing the DataFrame index as a column

print(f"DataFrame successfully exported to {output_file}")

DataFrame successfully exported to SF_Police_Reports_2003_to_2018_cleaned.csv
